In [2]:
%run ranker.py --data_path cmu-gpucloud-lsutawik/spr/solve_set_train --model_prefix Qwen-Qwen2.5 --task_prefix math --use_gcs

100%|██████████| 1920/1920 [00:36<00:00, 52.23it/s] 


In [27]:
model_list = df.model.unique()

for task in df.task.unique():
    print(f"Task: {task}")
    task_df = df[df.task == task]
    task_df = task_df.pivot_table(
        index=["idx"],
        columns=["model", "solve"],
        values=["accuracy", "output_tokens"],
        aggfunc="mean",
    )
    display(
        task_df
    )
    # pivot_table.columns = [':'.join(col).strip() for col in pivot_table.columns.values]
    # display(pivot_table)
    break

Task: math_algebra


accuracy                                                \
model Qwen-Qwen2.5-14B-Instruct           Qwen-Qwen2.5-32B-Instruct             
solve                        NL        PL                        NL        PL   
idx                                                                             
0                      1.000000  0.333333                  1.000000  1.000000   
1                      1.000000  1.000000                  1.000000  1.000000   
2                      0.000000  0.000000                  0.000000  0.000000   
3                      0.000000  0.000000                  0.000000  0.000000   
4                      1.000000  0.000000                  0.666667  0.000000   
..                          ...       ...                       ...       ...   
431                    1.000000  1.000000                  1.000000  1.000000   
432                    1.000000  0.666667                  1.000000  0.666667   
433                    0.000000  0.000000                  0.000000  0.000000   
434                    0.666667  0.333333                  0.333333  1.000000   
435                    0.333333  1.000000                  0.666667  1.000000   

                                                                              \
model Qwen-Qwen2.5-3B-Instruct           Qwen-Qwen2.5-72B-Instruct             
solve                       NL        PL                        NL        PL   
idx                                                                            
0                     0.666667  1.000000                  1.000000  0.666667   
1                     0.666667  0.000000                  1.000000  0.666667   
2                     0.000000  0.000000                  0.000000  0.000000   
3                     0.000000  0.000000                  0.000000  0.000000   
4                     0.333333  0.000000                  0.000000  0.000000   
..                         ...       ...                       ...       ...   
431                   0.333333  0.000000                  1.000000  1.000000   
432                   1.000000  0.666667                  1.000000  0.666667   
433                   0.000000  0.000000                  0.000000  0.000000   
434                   0.000000  0.333333                  0.666667  0.000000   
435                   0.333333  1.000000                  0.000000  1.000000   

                                                     output_tokens  \
model Qwen-Qwen2.5-7B-Instruct           Qwen-Qwen2.5-14B-Instruct   
solve                       NL        PL                        NL   
idx                                                                  
0                     1.000000  0.666667                658.333333   
1                     1.000000  0.000000                598.000000   
2                     0.000000  0.000000                883.000000   
3                     0.000000  0.000000                504.000000   
4                     1.000000  0.000000                687.333333   
..                         ...       ...                       ...   
431                   1.000000  1.000000                458.000000   
432                   1.000000  0.666667                570.000000   
433                   0.000000  0.000000                517.666667   
434                   0.333333  1.000000                573.000000   
435                   0.000000  1.000000               1528.000000   

                                                         \
model             Qwen-Qwen2.5-32B-Instruct               
solve          PL                        NL          PL   
idx                                                       
0      238.333333                534.333333  208.666667   
1      140.333333                652.000000  211.333333   
2      434.666667                841.666667  363.000000   
3      158.000000                464.000000  217.000000   
4      210.000000                654.333333  217.000000   
..            ...       

In [61]:
#                        model solve  accuracy  output_tokens
# 3  Qwen-Qwen2.5-32B-Instruct    PL  1.000000     208.666667
# 5   Qwen-Qwen2.5-3B-Instruct    PL  1.000000     348.333333
# 2  Qwen-Qwen2.5-32B-Instruct    NL  1.000000     534.333333
# 6  Qwen-Qwen2.5-72B-Instruct    NL  1.000000     640.666667
# 8   Qwen-Qwen2.5-7B-Instruct    NL  1.000000     655.333333
# 0  Qwen-Qwen2.5-14B-Instruct    NL  1.000000     658.333333
# 7  Qwen-Qwen2.5-72B-Instruct    PL  0.666667     216.000000
# 9   Qwen-Qwen2.5-7B-Instruct    PL  0.666667     233.666667
# 4   Qwen-Qwen2.5-3B-Instruct    NL  0.666667     650.333333
# 1  Qwen-Qwen2.5-14B-Instruct    PL  0.333333     238.333333

def rank_model_solution_pair(row):

    row = row.unstack(level=0).reset_index()
    row["size"] = row["model"].apply(lambda x: int(x.split("-")[2][:-1]))
    row["compute"] = 2 * row["size"] * row["output_tokens"]
    row.sort_values(by=['accuracy', 'compute'], ascending=[False, True])

    row["pair"] = row["model"] + "-" + row["solve"]

    return row["pair"].tolist()

# task_df[["value", "value"]] = task_df.apply(rank_model_solution_pair, axis=1)
task_df["rank"] = task_df.apply(rank_model_solution_pair, axis=1)


['Qwen-Qwen2.5-14B-Instruct-NL',
 'Qwen-Qwen2.5-14B-Instruct-PL',
 'Qwen-Qwen2.5-32B-Instruct-NL',
 'Qwen-Qwen2.5-32B-Instruct-PL',
 'Qwen-Qwen2.5-3B-Instruct-NL',
 'Qwen-Qwen2.5-3B-Instruct-PL',
 'Qwen-Qwen2.5-72B-Instruct-NL',
 'Qwen-Qwen2.5-72B-Instruct-PL',
 'Qwen-Qwen2.5-7B-Instruct-NL',
 'Qwen-Qwen2.5-7B-Instruct-PL']

In [ ]:
row = task_df.iloc[0]




,model,solve,accuracy,output_tokens,size,compute
5,Qwen-Qwen2.5-3B-Instruct,PL,1.000000,348.333333,3,2090.000000
8,Qwen-Qwen2.5-7B-Instruct,NL,1.000000,655.333333,7,9174.666667
3,Qwen-Qwen2.5-32B-Instruct,PL,1.000000,208.666667,32,13354.666667
0,Qwen-Qwen2.5-14B-Instruct,NL,1.000000,658.333333,14,18433.333333
2,Qwen-Qwen2.5-32B-Instruct,NL,1.000000,534.333333,32,34197.333333
6,Qwen-Qwen2.5-72B-Instruct,NL,1.000000,640.666667,72,92256.000000
9,Qwen-Qwen2.5-7B-Instruct,PL,0.666667,233.666667,7,3271.333333
4,Qwen-Qwen2.5-3B-Instruct,NL,0.666667,650.333333,3,3902.000000
7,Qwen-Qwen2.5-72B-Instruct,PL,0.666667,216.000000,72,31104.000000
1,Qwen-Qwen2.5-14B-Instruct,PL,0.333333,238.333333,14,6673.333333


In [54]:
row

               model                      solve
accuracy       Qwen-Qwen2.5-14B-Instruct  NL         1.000000
                                          PL         0.333333
               Qwen-Qwen2.5-32B-Instruct  NL         1.000000
                                          PL         1.000000
               Qwen-Qwen2.5-3B-Instruct   NL         0.666667
                                          PL         1.000000
               Qwen-Qwen2.5-72B-Instruct  NL         1.000000
                                          PL         0.666667
               Qwen-Qwen2.5-7B-Instruct   NL         1.000000
                                          PL         0.666667
output_tokens  Qwen-Qwen2.5-14B-Instruct  NL       658.333333
                                          PL       238.333333
               Qwen-Qwen2.5-32B-Instruct  NL       534.333333
                                          PL       208.666667
               Qwen-Qwen2.5-3B-Instruct   NL       650.333333
                      

In [52]:
line["pair"] = line["model"] + "-" + line["solve"]
line
# line.apply(lambda x: print(x[0]))

,model,solve,accuracy,output_tokens,pair
3,Qwen-Qwen2.5-32B-Instruct,PL,1.000000,208.666667,Qwen-Qwen2.5-32B-Instruct-PL
5,Qwen-Qwen2.5-3B-Instruct,PL,1.000000,348.333333,Qwen-Qwen2.5-3B-Instruct-PL
2,Qwen-Qwen2.5-32B-Instruct,NL,1.000000,534.333333,Qwen-Qwen2.5-32B-Instruct-NL
6,Qwen-Qwen2.5-72B-Instruct,NL,1.000000,640.666667,Qwen-Qwen2.5-72B-Instruct-NL
8,Qwen-Qwen2.5-7B-Instruct,NL,1.000000,655.333333,Qwen-Qwen2.5-7B-Instruct-NL
0,Qwen-Qwen2.5-14B-Instruct,NL,1.000000,658.333333,Qwen-Qwen2.5-14B-Instruct-NL
7,Qwen-Qwen2.5-72B-Instruct,PL,0.666667,216.000000,Qwen-Qwen2.5-72B-Instruct-PL
9,Qwen-Qwen2.5-7B-Instruct,PL,0.666667,233.666667,Qwen-Qwen2.5-7B-Instruct-PL
4,Qwen-Qwen2.5-3B-Instruct,NL,0.666667,650.333333,Qwen-Qwen2.5-3B-Instruct-NL
1,Qwen-Qwen2.5-14B-Instruct,PL,0.333333,238.333333,Qwen-Qwen2.5-14B-Instruct-PL


In [ ]:
import requests

def post_http_request(payload: dict, api_url: str) -> requests.Response:
    headers = {"User-Agent": "Test Client"}
    response = requests.post(api_url, headers=headers, json=payload)
    return response

r = post_http_request(
    payload = {
        "model": "jason9693/Qwen2.5-1.5B-apeach",
        "input": ["Hello, my name is"],
    },
    api_url = f"http://localhost:8100/classify"
    )

r.json()